In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import argparse
import itertools
import numpy as np
import os
import shutil
import tensorflow as tf
import cv2
import tqdm
import sys
sys.path.insert(0,'.')
sys.path.insert(0,'C:/Users/Sangwon Lee/Desktop/tensorpack')

import tensorpack.utils.viz as tpviz
from tensorpack.predict import MultiTowerOfflinePredictor, OfflinePredictor, PredictConfig
from tensorpack.tfutils import SmartInit, get_tf_version_tuple
from tensorpack.tfutils.export import ModelExporter
from tensorpack.utils import fs, logger

from dataset import DatasetRegistry, register_coco, register_balloon
from dataset.coco import register_DAGM
from config import config as cfg
from config import finalize_configs
from data import get_eval_dataflow, get_train_dataflow
from eval import DetectionResult, multithread_predict_dataflow, predict_image
from modeling.generalized_rcnn import ResNetC4Model, ResNetFPNModel
from viz import (
    draw_annotation, draw_final_outputs, draw_predictions,
    draw_proposal_recall, draw_final_outputs_blackwhite)

first = True

In [2]:
def do_visualize(model, model_path, nr_visualize=100, output_dir='output'):
    """
    Visualize some intermediate results (proposals, raw predictions) inside the pipeline.
    """
    df = get_train_dataflow()
    df.reset_state()

    pred = OfflinePredictor(PredictConfig(
        model=model,
        session_init=SmartInit(model_path),
        input_names=['image', 'gt_boxes', 'gt_labels'],
        output_names=[
            'generate_{}_proposals/boxes'.format('fpn' if cfg.MODE_FPN else 'rpn'),
            'generate_{}_proposals/scores'.format('fpn' if cfg.MODE_FPN else 'rpn'),
            'fastrcnn_all_scores',
            'output/boxes',
            'output/scores',
            'output/labels',
        ]))

    if os.path.isdir(output_dir):
        shutil.rmtree(output_dir)
    fs.mkdir_p(output_dir)
    with tqdm.tqdm(total=nr_visualize) as pbar:
        for idx, dp in itertools.islice(enumerate(df), nr_visualize):
            img, gt_boxes, gt_labels = dp['image'], dp['gt_boxes'], dp['gt_labels']

            rpn_boxes, rpn_scores, all_scores, \
                final_boxes, final_scores, final_labels = pred(img, gt_boxes, gt_labels)

            # draw groundtruth boxes
            gt_viz = draw_annotation(img, gt_boxes, gt_labels)
            # draw best proposals for each groundtruth, to show recall
            proposal_viz, good_proposals_ind = draw_proposal_recall(img, rpn_boxes, rpn_scores, gt_boxes)
            # draw the scores for the above proposals
            score_viz = draw_predictions(img, rpn_boxes[good_proposals_ind], all_scores[good_proposals_ind])

            results = [DetectionResult(*args) for args in
                       zip(final_boxes, final_scores, final_labels,
                           [None] * len(final_labels))]
            final_viz = draw_final_outputs(img, results)

            viz = tpviz.stack_patches([
                gt_viz, proposal_viz,
                score_viz, final_viz], 2, 2)

            if os.environ.get('DISPLAY', None):
                tpviz.interactive_imshow(viz)
            cv2.imwrite("{}/{:03d}.png".format(output_dir, idx), viz)
            pbar.update()


def do_evaluate(pred_config, output_file):
    print("do_evaluate")
    num_tower = max(cfg.TRAIN.NUM_GPUS, 1)
    graph_funcs = MultiTowerOfflinePredictor(
        pred_config, list(range(num_tower))).get_predictors()
    
#     print("graph_funcs[0]")
#     print("graph_funcs[0].input_tensors")
#     print(graph_funcs[0].input_tensors)
#     print("graph_funcs[0].output_tensors")
#     print(graph_funcs[0].output_tensors)
#     print("graph_funcs[0].return_input")
#     print(graph_funcs[0].return_input)
#     print("graph_funcs[0].sess")
#     print(graph_funcs[0].sess)
    
#     graph_funcs[0]
#     graph_funcs[0].input_tensors
#     [<tf.Tensor 'image:0' shape=(?, ?, 3) dtype=float32>]
#     graph_funcs[0].output_tensors
#     [<tf.Tensor 'tower0/output/boxes:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'tower0/output/scores:0' shape=(?,) dtype=float32>, <tf.Tensor 'tower0/output/labels:0' shape=(?,) dtype=int64>, <tf.Tensor 'tower0/output/masks:0' shape=(?, 28, 28) dtype=float32>]
#     graph_funcs[0].return_input
#     False
#     graph_funcs[0].sess
#     <tensorflow.python.client.session.Session object at 0x0000015C6BF0D388>

    for dataset in cfg.DATA.VAL:
        logger.info("Evaluating {} ...".format(dataset))
        dataflows = [
            get_eval_dataflow(dataset, shard=k, num_shards=num_tower)
            for k in range(num_tower)]
        print("dataflows")
        print(dataflows)
        all_results = multithread_predict_dataflow(dataflows, graph_funcs)
        output = output_file + '-' + dataset
        print("output")
        print(output)
        DatasetRegistry.get(dataset).eval_inference_results(all_results, output)


def do_predict(pred_func, input_file):
    img = cv2.imread(input_file, cv2.IMREAD_COLOR)
    results = predict_image(img, pred_func)
    if cfg.MODE_MASK:
        final = draw_final_outputs_blackwhite(img, results)
    else:
        final = draw_final_outputs(img, results)
    viz = np.concatenate((img, final), axis=1)
    cv2.imwrite("output.png", viz)
    logger.info("Inference output for {} written to output.png".format(input_file))
    tpviz.interactive_imshow(viz)

In [3]:
if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--load', help='load a model for evaluation.', required=True)
#     parser.add_argument('--visualize', action='store_true', help='visualize intermediate results')
#     parser.add_argument('--evaluate', help="Run evaluation. "
#                                            "This argument is the path to the output json evaluation file")
#     parser.add_argument('--predict', help="Run prediction on a given image. "
#                                           "This argument is the path to the input image file", nargs='+')
#     parser.add_argument('--benchmark', action='store_true', help="Benchmark the speed of the model + postprocessing")
#     parser.add_argument('--config', help="A list of KEY=VALUE to overwrite those defined in config.py",
#                         nargs='+')
#     parser.add_argument('--output-pb', help='Save a model to .pb')
#     parser.add_argument('--output-serving', help='Save a model to serving file')

#     args = parser.parse_args()
#     if args.config:
#         cfg.update_args(args.config)
        
    register_coco(cfg.DATA.BASEDIR)  # add COCO datasets to the registry
        
    # register_DAGM(cfg.DATA.BASEDIR)
#     MODEL = ResNetFPNModel() if cfg.MODE_FPN else ResNetC4Model()
    MODEL = ResNetFPNModel() 
    argsload = "C:/Users/Sangwon Lee/Desktop/tensorpack/train_log/maskrcnn_epch100/checkpoint"
    argsevaluate = "C:/Users/Sangwon Lee/Desktop/tensorpack/train_log/maskrcnn_epch100/stats.json"
    
    if not tf.test.is_gpu_available():
        from tensorflow.python.framework import test_util
        assert get_tf_version_tuple() >= (1, 7) and test_util.IsMklEnabled(), \
            "Inference requires either GPU support or MKL support!"
#     assert args.load
    finalize_configs(is_training=False)

#     if args.predict or args.visualize:
#         cfg.TEST.RESULT_SCORE_THRESH = cfg.TEST.RESULT_SCORE_THRESH_VIS

#     if args.visualize:
#         do_visualize(MODEL, args.load)
#     else:
    predcfg = PredictConfig(
        model=MODEL,
        session_init=SmartInit(argsload),
        input_names=MODEL.get_inference_tensor_names()[0],
        output_names=MODEL.get_inference_tensor_names()[1])

#     if args.output_pb:
#         ModelExporter(predcfg).export_compact(args.output_pb, optimize=False)
#     elif args.output_serving:
#         ModelExporter(predcfg).export_serving(args.output_serving, optimize=False)

#     if args.predict:
#         predictor = OfflinePredictor(predcfg)
#         for image_file in args.predict:
#             do_predict(predictor, image_file)
#     elif args.evaluate:
#     assert args.evaluate.endswith('.json'), args.evaluate
    do_evaluate(predcfg, argsevaluate)
#     elif args.benchmark:
#         df = get_eval_dataflow(cfg.DATA.VAL[0])
#         df.reset_state()
#         predictor = OfflinePredictor(predcfg)
#         for _, img in enumerate(tqdm.tqdm(df, total=len(df), smoothing=0.5)):
#             # This includes post-processing time, which is done on CPU and not optimized
#             # To exclude it, modify `predict_image`.
#             predict_image(img[0], predictor)


register_coco
[0626 14:26:42 @config.py:335] Config: ------------------------------------------
{'BACKBONE': {'FREEZE_AFFINE': False,
              'FREEZE_AT': 2,
              'NORM': 'FreezeBN',
              'RESNET_NUM_BLOCKS': [3, 4, 6, 3],
              'STRIDE_1X1': False,
              'TF_PAD_MODE': False,
              'WEIGHTS': 'ImageNet-R50-AlignPadding.npz'},
 'CASCADE': {'BBOX_REG_WEIGHTS': [[10.0, 10.0, 5.0, 5.0], [20.0, 20.0, 10.0, 10.0],
                                  [30.0, 30.0, 15.0, 15.0]],
             'IOUS': [0.5, 0.6, 0.7]},
 'DATA': {'ABSOLUTE_COORD': True,
          'BASEDIR': 'C:/Users/Sangwon Lee/Desktop/COCO',
          'CLASS_NAMES': ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                          'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
                          'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                          'elephant', 'bear', 'zebra', 'g

[0626 14:26:42 @registry.py:90] 'group1/block3/conv3': [1, 128, ?, ?] --> [1, 512, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block0/conv1': [1, 512, ?, ?] --> [1, 256, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block0/conv2': [1, 256, ?, ?] --> [1, 256, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block0/conv3': [1, 256, ?, ?] --> [1, 1024, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block0/convshortcut': [1, 512, ?, ?] --> [1, 1024, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block1/conv1': [1, 1024, ?, ?] --> [1, 256, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block1/conv2': [1, 256, ?, ?] --> [1, 256, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block1/conv3': [1, 256, ?, ?] --> [1, 1024, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block2/conv1': [1, 1024, ?, ?] --> [1, 256, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block2/conv2': [1, 256, ?, ?] --> [1, 256, ?, ?]
[0626 14:26:42 @registry.py:90] 'group2/block2/conv3': [1, 256, ?, ?] --> [1, 1024, ?, ?]
[0626 14

[0626 14:26:44 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, learning_rate
[0626 14:26:45 @sessinit.py:114] Restoring checkpoint from C:/Users/Sangwon Lee/Desktop/tensorpack/train_log/maskrcnn_epch100\model-50000 ...
INFO:tensorflow:Restoring parameters from C:/Users/Sangwon Lee/Desktop/tensorpack/train_log/maskrcnn_epch100\model-50000
[0626 14:26:45 @<ipython-input-2-606d21de61ab>:80] Evaluating coco_val2017 ...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[0626 14:26:45 @coco.py:61] Instances loaded from C:/Users/Sangwon Lee/Desktop/COCO\annotations/instances_val2017.json.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

[0626 14:26:45 @timer.py:45] Load annotations for instances_val2017.json finished, time:0.0020 sec.
[0626 14:26:45 @data.py:393] get_eval_dataflowcoco_val2017
[0626 14:26:45 @data.py:394] Found 1 images for inference.



  0%|                                                                                               |0/1[00:00<?,?it/s]

dataflows
predict_dataflow
predict_image
model_func


100%|#######################################################################################|1/1[00:02<00:00, 0.37it/s]

boxes : [[  0.       111.33264  800.       874.      ]
 [126.91217  128.41095  800.       874.      ]
 [  0.        66.31091  545.32153  871.6614  ]
 [  0.       186.59998  800.       856.7649  ]
 [ 28.814087  51.62668  745.7541   535.5835  ]
 [  0.        97.963165 372.49066  874.      ]
 [232.25693   93.01294  800.       615.8345  ]
 [ 12.364578 312.4598   373.7177   874.      ]
 [  0.       312.15137  377.17862  872.0852  ]
 [  6.551941  70.18579  790.0018   698.11304 ]
 [ 22.516693  64.27887  651.3352   423.1701  ]]
probs : [0.6834891  0.51842    0.21731064 0.20058276 0.12538123 0.10867176
 0.10091011 0.10016387 0.09358117 0.05339874 0.05242478]
labels : [22 16 16 17 16 22 22 16 17 19 22]
masks : [array([[[1.79542005e-02, 2.60300040e-02, 3.37189138e-02, ...,
         5.02315164e-03, 2.10858285e-02, 2.67854929e-02],
        [1.65822804e-02, 2.28801072e-02, 3.32157910e-02, ...,
         3.93405557e-03, 1.60883069e-02, 2.46198773e-02],
        [6.75287843e-03, 1.23693943e-02, 4.683932

coco.py eval_inference_results
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 1.000
 Average Recall     (AR) @[ IoU=0.5